<a href="https://colab.research.google.com/github/kai-2320181/Prog2_11th/blob/main/work3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
第１４回課題提出用